In [ ]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import src.functions as src


fs=500
freq_bands={'delta' : [1.5,3.5],
            'theta' : [4,8],
            'alpha' : [8.5, 12],
            'beta' : [13,35],
            'high' : [35, 45],
            'all': [1.5,45],
            'low': [1.5,12]}


data = {}
labels = {}
file= [item for item in os.listdir('data') if item.endswith('set')]

for file_index, file_name in enumerate(file):
    path_ = os.path.join( 'data', file_name )
    dados = mne.read_epochs_eeglab(path_)
    channel_names = dados.info['ch_names']
    subject_id = file_name[:3]
    
    # Extract labels
    Corr_labels = len(dados['proCorr','antiCorr'].get_data())
    Err_labels = len(dados['proErr','antiErr','nogoErr'].get_data())
    labels[subject_id] = np.array([0] * Corr_labels + [1] * Err_labels)
    
    # Initialize channel data dictionary
    channel_data = {}
    
    # Iterate over 'proCorr' and 'antiCorr' events
    for epoch_values in dados['proCorr','antiCorr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    # Iterate over 'proErr', 'antiErr', and 'nogoErr' events
    for epoch_values in dados['proErr','antiErr','nogoErr'].get_data():
        for channel_index, channel_values in enumerate(epoch_values):
            channel_name = channel_names[channel_index]
            if channel_name not in channel_data:
                channel_data[channel_name] = []
            channel_data[channel_name].append(list(channel_values))
    
    data[subject_id] = channel_data


In [ ]:
def data_pre_post(data):
    pre_response={}
    post_response={}
    for subject_id, dados in data.items():
        channel_data_pre={}
        channel_data_post={}
        for channel_name, values in dados.items():
            values_pre=[]
            values_post=[]
            for i in range(len(values)):
                
                index= int(len(values[i])/2)
                values_pre.append(values[i][0:index])
                values_post.append(values[i][index:-1])
            channel_data_pre[channel_name]= values_pre
            channel_data_post[channel_name]= values_post
            
        pre_response[subject_id] = channel_data_pre #(14,60,n_events,250)
        post_response[subject_id] = channel_data_post
    return pre_response, post_response

pre_response, post_response = data_pre_post(data)

In [ ]:
f_pre,S_pre= src.getpsd(pre_response,fs)    # S_pre ->(14,60,n_events,n_points)
f_post,S_post= src.getpsd(post_response,fs)

In [ ]:
pre_data_per_channel= src.getdataperchannel(S_pre) #(subject_id,60,n_events,n_points)
post_data_per_channel= src.getdataperchannel(S_post)

In [ ]:
from importlib import reload
reload(src)

### FCZ cluster 

In [ ]:
data1= src.extend_list(src.feature('fcz_features','.','.','theta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post, f_post))
data2= src.extend_list(src.feature('fcz_features','.','.','theta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
data3= src.extend_list(src.feature('fcz_features','.','.','theta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data4= src.extend_list(src.feature('fcz_features','.','.','theta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data5= src.extend_list(src.feature('fcz_features','.','.','theta','other',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
data6= src.extend_list(src.feature('fcz_features','.','.','theta','other',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
data7= src.extend_list(src.feature('fcz_features','.','.','theta','theta',post_data_per_channel,pre_data_per_channel,freq_bands,f_post,f_pre))

### All

In [ ]:
data8={}
data9={}
data10={}
data11={}
data12={}
data13={}
data14={}
data15={}
data16={}
for channel_name in post_data_per_channel['P01'].keys():    
    feature8= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature9= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature10= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature11= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature12= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','theta',post_data_per_channel,pre_data_per_channel,freq_bands,f_post,f_pre))
    feature13= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature14= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature15= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature16= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data8[channel_name]=feature8
    data9[channel_name]=feature9
    data10[channel_name]=feature10
    data11[channel_name]=feature11
    data12[channel_name]=feature12
    data13[channel_name]=feature13
    data14[channel_name]=feature14
    data15[channel_name]=feature15
    data16[channel_name]=feature16

### Midfrontal

In [ ]:
data17={}
data18={}
for channel_name in post_data_per_channel['P01'].keys():
    feature17= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature18= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data17[channel_name]=feature17
    data18[channel_name]=feature18

### Low

In [ ]:
data19={}
data20={}
data21={}
data22={}
data23={}
data24={}
data25={}
data26={}
data27={}
data28={}
data29={}
data30={}
for channel_name in post_data_per_channel[subject_id].keys():        
    
    feature19= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature20= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature21= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature22= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature23= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature24= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',post_data_per_channel,post_data_per_channel,freq_bands,f_post,f_post))
    feature25= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature26= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature27= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature28= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature29= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    feature30= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',pre_data_per_channel,pre_data_per_channel,freq_bands,f_pre,f_pre))
    data19[channel_name]=feature19
    data20[channel_name]=feature20
    data21[channel_name]=feature21
    data22[channel_name]=feature22
    data23[channel_name]=feature23
    data24[channel_name]=feature24
    data25[channel_name]=feature25
    data26[channel_name]=feature26
    data27[channel_name]=feature27
    data28[channel_name]=feature28
    data29[channel_name]=feature29
    data30[channel_name]=feature30

### Temporal

In [ ]:
data31={}
data32={}
FCZ_Cluster=['FZ','FC1','FCZ','FC2','CZ']
PE_Cluster=['CPZ','P1','PZ','P2','POZ']
for subject_id,values in post_response.items():
    mean_all70_160_values = []
    mean_all200_500_values = []
    for i in range(len(next(iter(values.values())))):
        all70_160_values = []
        all200_500_values = []
        for channel_name, channel_values in values.items():   
            if channel_name in FCZ_Cluster:         
                all70_160_values.append(np.mean(channel_values[i][35:81]))
            elif channel_name in PE_Cluster:
                all200_500_values.append(np.mean(channel_values[i][100:-1]))
            else:
                continue

        mean_all70_160_values.append(np.mean(all70_160_values))
        mean_all200_500_values.append(np.mean(all200_500_values))
    data31[subject_id]= mean_all70_160_values
    data32[subject_id]= mean_all200_500_values

data31=src.extend_list(data31)
data32=src.extend_list(data32)

In [ ]:
data33 = {}
data34 = {}
# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in pre_response.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in data33:
            data33[channel_name] = []
        if channel_name not in data34:
            data34[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_500_250 = np.mean(channel_values[i][0:126])
            mean_250_0 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            data33[channel_name].append(mean_500_250)
            data34[channel_name].append(mean_250_0)


In [ ]:
data35 = {}
data36 = {}
# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in post_response.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in data35:
            data35[channel_name] = []
        if channel_name not in data36:
            data36[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_0_250 = np.mean(channel_values[i][0:126])
            mean_250_500 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            data35[channel_name].append(mean_0_250)
            data36[channel_name].append(mean_250_500)

### Saving Features

In [ ]:
import os
import pandas as pd

# Define the file path
file_path = 'dataset1_features.csv'

# Check if the file exists and remove it
if os.path.exists(file_path):
    os.remove(file_path)

# Create an empty list to store the dictionaries
dicts = []

# Iterate over the range to collect data dictionaries
for feature_index in range(1, 37):
    feature_corr_key = 'data{}'.format(feature_index)
    dicts.append(globals().get(feature_corr_key, {}))

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate through each dictionary/list and its index
for index, feature_data in enumerate(dicts, start=1):
    # Check if feature_data is a dictionary
    if isinstance(feature_data, dict):
        # Filter out channels with missing or empty data
        filtered_dict = {channel: data for channel, data in feature_data.items() if data}

        # Convert the filtered dictionary to a DataFrame
        if filtered_dict:
            try:
                print(f"Processing dictionary at index {index}")
                df = pd.DataFrame(filtered_dict)
                # Rename the columns to include the iteration number
                df.columns = [f'{index}_{col}' for col in df.columns]
                # Concatenate the DataFrame to the combined DataFrame
                combined_df = pd.concat([combined_df, df], axis=1)
            except Exception as e:
                print(f"An error occurred for dictionary at index {index}: {e}")
                continue
    # Check if feature_data is a list
    elif isinstance(feature_data, list):
        if feature_data:
            try:
                print(f"Processing list at index {index}")
                df = pd.DataFrame({f'{index}_data': feature_data})
                # Concatenate the DataFrame to the combined DataFrame
                combined_df = pd.concat([combined_df, df], axis=1)
            except Exception as e:
                print(f"An error occurred for list at index {index}: {e}")
                continue
    else:
        print(f"Skipping index {index} as it is neither a dictionary nor a list.")

# Extend the labels list and add it as a new column in the combined DataFrame
combined_df['labels'] = src.extend_list(labels)

# Save the combined DataFrame to a CSV file
combined_df.to_csv(file_path, index=False)
